Importing all the necessary libraries for the ELT

In [1]:
import pandas as pd

In [2]:
import pyorc

In [3]:
from datetime import datetime

Read file

In [4]:
example = open("https://storage.cloud.google.com/trades-ivan-test/trades.orc", 'rb')

OSError: [Errno 22] Invalid argument: 'https://storage.cloud.google.com/trades-ivan-test/trades.orc'

In [ ]:
reader = pyorc.Reader(example)

In [ ]:
schema = reader.schema

In [ ]:
columns = schema.fields

In [ ]:
df=pd.DataFrame(reader, columns=columns, dtype=str)

In [ ]:
df

In [ ]:
print(df.info())

Prepare the DataFreame


In [ ]:
df = df.rename(columns={'trade_id': 'trade_id_src'})

In [ ]:
df = df.rename(columns={'price': 'price_src'})

In [ ]:
df = df.rename(columns={'quantity': 'quantity_src'})

In [ ]:
df = df.rename(columns={'time': 'time_src'})

In [ ]:
df = df.rename(columns={'isSelling': 'isSelling_src'})

Clean and customize

In [ ]:
df['trade_id_str'] = df['trade_id_src'].str.slice(stop=-2)

In [ ]:
df['price_fl'] = df['price_src'].astype(float)

In [ ]:
df['quantity_fl'] = df['quantity_src'].astype(float)

In [ ]:
df['time_dt'] = pd.to_datetime(df['time_src'].astype(float)/1000, unit='s')


In [ ]:
df['isSelling_bl'] = df['isSelling_src'].str.lower() == 'true'

In [ ]:
df = df.sort_values('time_dt')

In [ ]:
df = df.dropna()

In [ ]:
df = df.drop_duplicates(subset=['price_fl', 'quantity_fl', 'time_dt', 'isSelling_bl' ], keep='first')

In [ ]:
df

In [ ]:
print(df.info())

Index and Audit fields

In [ ]:
df['last_modified_dt'] = datetime.now()

In [ ]:
df = df.set_index('trade_id_str')

In [ ]:
df

In [ ]:
print(df.info())

Find Metrics

In [ ]:
df['total_fl'] = df.apply(lambda row: -1 if row['isSelling_bl'] == False else 1, axis=1) * df['quantity_fl'].values * df['price_fl']

In [ ]:
bot_outcome = df['total_fl'].sum()

In [ ]:
df['rowcount_in'] = df['isSelling_bl'].where(df['isSelling_bl'], 0).astype(int).groupby((df['isSelling_bl'] != df['isSelling_bl'].shift()).cumsum()).cumsum()

In [ ]:
longest_chain = pd.to_numeric(df['rowcount_in'].max())

In [ ]:
max_index = pd.to_numeric(df['rowcount_in'].idxmax())

In [ ]:
min_index = (max_index-longest_chain).astype(str)

In [ ]:
max_index = max_index.astype(str)

In [ ]:
chain_price_diff = (df.loc[max_index, 'quantity_fl']*df.loc[max_index, 'price_fl'])-(df.loc[min_index, 'quantity_fl']*df.loc[min_index, 'price_fl'])

Results

In [ ]:
df

In [ ]:
print(df.info())

In [ ]:
print('Bot outcome for the day:', bot_outcome)
print('Longest chain of consecutive sells:', longest_chain)
print('Price difference between first and last operation of longest chain:', chain_price_diff)